# CuPy

现在我们已经使用Numba探索了一些底层GPU API，让我们换个方向，使用[CuPy](https://cupy.dev/)来处理一些高级数组功能。

CuPy有来自包括NVIDIA在内的许多组织的维护者。CuPy实现了熟悉的NumPy API，但后端是用CUDA C++编写的。这使得已经熟悉NumPy的人只需切换导入语句就能快速获得GPU加速。

In [1]:
import numpy as np
import cupy as cp
cp.cuda.Stream.null.synchronize()

让我们通过这篇博客文章中的一些简单示例来学习：https://towardsdatascience.com/heres-how-to-use-cupy-to-make-numpy-700x-faster-4b920dda1f56

## 创建数组

首先让我们在CPU和GPU上分别创建一个`2GB`的数组，并比较所需的时间。

In [2]:
%%timeit -r 1 -n 10
global x_cpu
x_cpu = np.ones((1000, 500, 500))

864 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


In [3]:
%%timeit -n 10
global x_gpu
x_gpu = cp.ones((1000, 500, 500))

cp.cuda.Stream.null.synchronize()

10.7 ms ± 5.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


_注意：为了使计时公平，我们需要在这里显式调用`cp.cuda.Stream.null.synchronize()`。默认情况下，cupy会并发运行GPU代码，函数会在GPU完成之前退出。调用`synchronize()`使我们在返回之前等待GPU完成。_

我们可以看到，在GPU上创建这个数组比在CPU上快得多，但这次我们的代码看起来完全相同。我们不必担心内核、线程、块或任何这些东西。

## 基本操作

接下来让我们看看如何对数组进行一些数学运算。我们可以从将数组中的每个值乘以`5`开始。

In [4]:
%%time
x_cpu *= 5

CPU times: user 173 ms, sys: 878 μs, total: 174 ms
Wall time: 174 ms


In [5]:
%%time
x_gpu *= 5

cp.cuda.Stream.null.synchronize()

CPU times: user 103 ms, sys: 11 ms, total: 114 ms
Wall time: 113 ms


同样，GPU完成得更快，但代码保持不变。

现在让我们按顺序执行几个操作，在我们的Numba示例中，如果没有显式的内存管理，这些操作会受到内存传输时间的影响。

In [6]:
%%time
x_cpu *= 5
x_cpu *= x_cpu
x_cpu += x_cpu

CPU times: user 500 ms, sys: 0 ns, total: 500 ms
Wall time: 499 ms


In [7]:
%%time
x_gpu *= 5
x_gpu *= x_gpu
x_gpu += x_gpu

cp.cuda.Stream.null.synchronize()

CPU times: user 198 ms, sys: 10.7 ms, total: 209 ms
Wall time: 208 ms


同样，我们可以看到即使没有显式管理内存，GPU运行得也快得多。这是因为CuPy为我们透明地处理了所有这些。

## 更复杂的操作

现在我们已经尝试了一些运算符，让我们深入了解一些NumPy函数。让我们比较在一个稍小的数据数组上运行奇异值分解。

In [8]:
%%time
x_cpu = np.random.random((1000, 1000))
u, s, v = np.linalg.svd(x_cpu)

CPU times: user 55.5 s, sys: 4.69 s, total: 1min
Wall time: 3.13 s


In [9]:
%%time
x_gpu = cp.random.random((1000, 1000))
u, s, v = cp.linalg.svd(x_gpu)

CPU times: user 596 ms, sys: 325 ms, total: 921 ms
Wall time: 2.09 s


正如我们所看到的，GPU再次以完全相同的API优于CPU。

这里还有一个有趣的地方值得注意，NumPy可以智能地分派这样的函数调用。在上面的例子中，我们调用了`cp.linalg.svd`，但我们也可以调用`np.linalg.svd`并传递我们的GPU数组。NumPy会检查输入并代表我们调用`cp.linalg.svd`。这使得在代码中引入`cupy`变得更加容易，只需最少的更改。

In [10]:
%%time
x_gpu = cp.random.random((1000, 1000))
u, s, v = np.linalg.svd(x_gpu)  # Note the `np` used here

cp.cuda.Stream.null.synchronize()

CPU times: user 460 ms, sys: 198 ms, total: 658 ms
Wall time: 657 ms


## 设备管理

CuPy有一个当前设备的概念，它是进行数组分配、操作、计算等的默认GPU设备。假设当前设备的ID是`0`。在这种情况下，以下代码将在GPU 0上创建一个数组`x_on_gpu0`。

In [11]:
with cp.cuda.Device(0):
   x_on_gpu0 = cp.random.random((100000, 1000))

x_on_gpu0.device

<CUDA Device 0>

一般来说，CuPy函数期望数组与当前设备在同一设备上。传递存储在非当前设备上的数组可能会根据硬件配置而工作，但通常不建议这样做，因为它可能不具有良好的性能。